# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [20]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [39]:
def extract_time_series_features(data, window_size=60):
    """Extract rolling window features from time series data.
    
    Parameters
    ----------
    data : pd.DataFrame
        Preprocessed physiological data
    window_size : int
        Size of the rolling window in seconds
        
    Returns
    -------
    results : pd.DataFrame
        DataFrame containing extracted features for each signal
    """
    # 1. Calculate rolling window statistics
    # 2. Include mean, std, min, max, and autocorrelation
    data = data.copy()
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Set timestamp as index for time-based rolling
    data = data.set_index('timestamp')

    # Function to compute autocorrelation using the pd Series.autocorr()
    def autocorr(x, lag=1):
        if len(x) <= lag:
            return np.nan
        return x.autocorr(lag=lag)

    features = []

    # Group by subject and session
    grouped = data.groupby(['subject_id', 'session'])

    # because I sampled 4 Hz
    window_size = 4 * window_size

    for (subject, session), group in grouped:
        # Apply rolling window
        rolling = group[['eda', 'heart_rate', 'temperature']].rolling(f'{window_size}s')

        # Compute features
        stats = pd.DataFrame({
            'eda_mean': rolling['eda'].mean(),
            'eda_std': rolling['eda'].std(),
            'eda_min': rolling['eda'].min(),
            'eda_max': rolling['eda'].max(),
            'eda_autocorr': rolling['eda'].apply(lambda x: autocorr(x), raw=False),
            'heart_rate_mean': rolling['heart_rate'].mean(),
            'heart_rate_std': rolling['heart_rate'].std(),
            'heart_rate_min': rolling['heart_rate'].min(),
            'heart_rate_max': rolling['heart_rate'].max(),
            'heart_rate_autocorr': rolling['heart_rate'].apply(lambda x: autocorr(x), raw=False),
            'temperature_mean': rolling['temperature'].mean(),
            'temperature_std': rolling['temperature'].std(),
            'temperature_min': rolling['temperature'].min(),
            'temperature_max': rolling['temperature'].max(),
            'temperature_autocorr': rolling['temperature'].apply(lambda x: autocorr(x), raw=False),
        })

        # Restore identifiers
        stats['subject_id'] = subject
        stats['session'] = session

        features.append(stats)

    result = pd.concat(features).reset_index()
    return result

In [40]:
def load_subject_data(subject_id, base_dir='data/processed'):
    filename = f"{subject_id}.csv"  # e.g., S1.csv
    file_path = os.path.join(base_dir, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"No file found for subject {subject_id} at {file_path}")
    
    return pd.read_csv(file_path)

data_dir = 'data/processed'
raw_data_path = os.path.join(os.getcwd(), data_dir)

df_s1 = load_subject_data("S1")
print(df_s1.head())

df_s1_cut = df_s1.head(5000)

rolling = extract_time_series_features(df_s1_cut, window_size=60)
print(rolling.head(10))

# gets some warnings when the 

                 timestamp       eda  heart_rate  temperature subject_id  \
0  2018-12-05 16:29:07.000  0.024345  116.000000        21.91         S1   
1  2018-12-05 16:29:07.250  0.023064  107.625004        21.91         S1   
2  2018-12-05 16:29:07.500  0.024345   99.250000        21.91         S1   
3  2018-12-05 16:29:07.750  0.023064   90.874996        21.91         S1   
4  2018-12-05 16:29:08.000  0.023064   82.500000        21.93         S1   

  session  
0   Final  
1   Final  
2   Final  
3   Final  
4   Final  


C:\Users\kevxs\anaconda3\lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)


                timestamp  eda_mean   eda_std   eda_min   eda_max  \
0 2018-12-05 16:29:07.000  0.024345       NaN  0.024345  0.024345   
1 2018-12-05 16:29:07.250  0.023704  0.000906  0.023064  0.024345   
2 2018-12-05 16:29:07.500  0.023918  0.000740  0.023064  0.024345   
3 2018-12-05 16:29:07.750  0.023704  0.000740  0.023064  0.024345   
4 2018-12-05 16:29:08.000  0.023576  0.000702  0.023064  0.024345   
5 2018-12-05 16:29:08.250  0.023491  0.000662  0.023064  0.024345   
6 2018-12-05 16:29:08.500  0.023613  0.000685  0.023064  0.024345   
7 2018-12-05 16:29:08.750  0.023544  0.000663  0.023064  0.024345   
8 2018-12-05 16:29:09.000  0.023491  0.000640  0.023064  0.024345   
9 2018-12-05 16:29:09.250  0.023320  0.000810  0.021783  0.024345   

   eda_autocorr  heart_rate_mean  heart_rate_std  heart_rate_min  \
0           NaN       116.000000             NaN      116.000000   
1           NaN       111.812502        5.922016      107.625004   
2     -1.000000       107.625001    

## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [34]:
def build_arima_model(series, order=(1,1,1), output_dir='plots'):
    """Fit an ARIMA model to the time series and generate diagnostic plots.
    
    Parameters
    ----------
    series : pd.Series
        Time series data to model
    order : tuple
        (p,d,q) order of the ARIMA model
    output_dir : str
        Directory to save diagnostic plots
        
    Returns
    -------
    model_fit: statsmodels.tsa.arima.model.ARIMAResults
        Fitted ARIMA model
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # note that in this project we don't actually separate out things by session, so naively throwing in a "series" from previous functions will probably mix "final" and "midterms" data
    
    # Your code here
    # 1. Fit ARIMA model

    # Step 1: Check for stationarity using Augmented Dickey-Fuller test
    result = adfuller(series.dropna())
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"p-value: {result[1]:.4f}")
    if result[1] > 0.05:
        print("Warning: Series appears non-stationary (p > 0.05).")

    series = series.reset_index(drop=True) # since the series isn't including timestamp data

    # Step 2: Fit ARIMA model
    model = ARIMA(series, order=order) # not actually sure what the best order to choose is - can maybe run ADF test afterwards, but we actually set the order as an argument so just use the set order regardless
    model_fit = model.fit()

    # Step 3: Plot model fit
    plt.figure(figsize=(10, 4))
    plt.plot(series, label='Original')
    plt.plot(model_fit.fittedvalues, label='Fitted', color='orange')
    plt.title('ARIMA Model Fit')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'arima_fit.png'))
    plt.close()

    # Step 4: Plot residuals
    residuals = model_fit.resid
    fig, ax = plt.subplots(2, 1, figsize=(10, 6))
    ax[0].plot(residuals)
    ax[0].set_title('Residuals')
    ax[1].hist(residuals, bins=30)
    ax[1].set_title('Residual Distribution')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'arima_residuals.png'))
    plt.close()

    # Step 5: Forecast plot
    forecast_steps = 50
    forecast = model_fit.get_forecast(steps=forecast_steps)
    # forecast_index = pd.date_range(start=series.index[-1], periods=forecast_steps + 1, freq='250ms')[1:]
    forecast_index = range(len(series), len(series) + forecast_steps)
    forecast_series = pd.Series(forecast.predicted_mean.values, index=forecast_index)

    plt.figure(figsize=(10, 4))
    plt.plot(series, label='Original')
    plt.plot(forecast_series, label='Forecast', color='orange')
    plt.title('ARIMA Forecast')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'arima_forecast.png'))
    plt.close()

    return model_fit

In [38]:
build_arima_model(df_s1.head(10000)['eda'], order=(1,1,1), output_dir='plots/S1_eda_arima')

ADF Statistic: -1.3962
p-value: 0.5841
